In [1]:
import pandas as pd
import math
import numpy as np
import unicodedata
import re
# import awswrangler as wr

In [2]:
def convert_and_extract(value):
    if pd.notna(value):
        value = value.replace(',', '.')[:8]
        return value
    else:
        return np.nan

In [3]:
df_clientes = pd.read_parquet('/home/pydev/workflow/dt_unifed_statistics/data/clientes_b2b.parquet')
# df_clientes = [(df_clientes['COD_CANAL']=="ON") & (df_clientes['COD_CANAL']=="OFF")]
df_clientes = df_clientes[(df_clientes['COD_CANAL'] == 'OFF') | (df_clientes['COD_CANAL'] == 'ON')] 
df_clientes = df_clientes[(df_clientes['IND_ESTADO_ACTIVO_CLIENTE'] == 1)] 
df_clientes["LONGITUD"] = df_clientes['LONGITUD'].apply(convert_and_extract)
df_clientes["LATITUD"]  = df_clientes['LATITUD'].apply(convert_and_extract)
df_clientes['LONGITUD']   = pd.to_numeric(df_clientes['LONGITUD'], errors='coerce')
df_clientes['LATITUD']    = pd.to_numeric(df_clientes['LATITUD'], errors='coerce')
df_clientes.tail(3)
# df_clientes.info()

# df_clientes[df_clientes['COD_VENDEDOR']== '0102']

,COD_CIA,ROWID_TERCERO,NIT_TERCERO,ID_TERCERO,RAZON_SOCIAL,COD_SUCURSAL_CLIENTE,DES_SUCURSAL_CLIENTE,IND_ESTADO_ACTIVO_CLIENTE,COD_DIA_VISITA,DES_DIA_VISITA,COD_CO,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,COD_VENDEDOR,DES_VENDEDOR,COD_CANAL,LONGITUD,LATITUD
96997,2,633820.0,700200366,700200366-001,CARLOS GUSTAVO ROSALES RAMMIREZ,001,SUSHI BY CARLOS RAMIREZ,1,000,No determinado,001,000,No determinado,0115,VEND ARANGO VELEZ CAROLINA,ON,NaN,NaN
97000,2,633845.0,901747029,901747029-001,PROVINCIA JYE,001,PROVINCIA JYE,1,006,Viernes,014,003,Quincena 2 y 4,0254,VEND AUTOV SIMANCA MARTINEZ JHONNY DARIO,OFF,NaN,NaN
97001,2,633949.0,1063148012,1063148012-001,MERCADO GOMEZ JHONNY JUNIOR,001,ZUMBATE LA PIKOTERA,1,002,Lunes,014,002,Quincena 1 y 3,0254,VEND AUTOV SIMANCA MARTINEZ JHONNY DARIO,OFF,NaN,NaN


In [4]:
df_calendario = pd.read_parquet('/home/pydev/workflow/dt_unifed_statistics/data/calendario_visitas.parquet')
df_calendario = df_calendario.iloc[:, [0, 1, 3, 5, 7]]
# Usar el método .melt() para "desapilar" las columnas de códigos
df_calendario = pd.melt(df_calendario, id_vars=["FECHA", "COD_DIA VISITA"], value_vars=["COD SEMANAL",	"COD QUINCENAL","COD MENSUAL"], var_name="Tipo_Codigo", value_name="Codigo_Frecuencia")
# Cambiar el nombre de la columna utilizando el método .rename()
df_calendario.rename(columns={'COD_DIA VISITA': 'COD_DIA_VISITA','Codigo_Frecuencia':'COD_FRECUENCIA_VISITA', 'FECHA': 'FECHAS_VISITA', 'Tipo_Codigo':'TIPO_CODIGO'}, inplace=True)
df_calendario['FECHAS_VISITA'] = pd.to_datetime(df_calendario['FECHAS_VISITA'], format= "%d/%m/%Y").astype(str)
# df_calendario["type_2"] = "calendario_visitas"
df_calendario = df_calendario.iloc[:, [0, 1, 3]]
df_calendario.head(3)
# df_calendario.info()

,FECHAS_VISITA,COD_DIA_VISITA,COD_FRECUENCIA_VISITA
0,2023-01-01,001,001
1,2023-01-02,002,001
2,2023-01-03,003,001


In [5]:
print(df_clientes.shape)
print(df_calendario.shape)

(22523, 18)
(1095, 3)


In [6]:
# Merge
base_clientes_rutero = pd.merge(df_clientes, df_calendario)
# base_clientes_rutero.head(10)
base_clientes_rutero.shape
# base_clientes_rutero.to_csv('clientes_rut.csv')

(561325, 19)

In [7]:
df_visitas = pd.read_parquet('/home/pydev/workflow/dt_unifed_statistics/data/visitas_vendedores.parquet')
df_visitas['FECHAS_VISITA']     = pd.to_datetime(df_visitas['Inicio_Visita'])
df_visitas["Logintud"]          = df_visitas['Logintud'].apply(convert_and_extract)
df_visitas["Latitud"]           = df_visitas['Latitud'].apply(convert_and_extract)
df_visitas['AÑO']               = df_visitas['FECHAS_VISITA'].dt.year.astype(str)
df_visitas                      = df_visitas[df_visitas["AÑO"]=='2023']
df_visitas['FECHAS_VISITA']     = df_visitas['FECHAS_VISITA'].dt.date.astype(str)
df_visitas['FECHAS_VISITA_V']   = df_visitas['FECHAS_VISITA']
df_visitas                      = df_visitas.iloc[:, [5, 0, 1, 2, 3, 10, 12]]
df_visitas.head()
# df_visitas.info()

,ID_TERCERO,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,FECHAS_VISITA,FECHAS_VISITA_V
109,1070597717-001,-74.1781,4.764811,2023-01-04 08:33:43,NaT,2023-01-04,2023-01-04
110,1004120709-001,-74.8264,11.03748,2023-01-14 09:29:36,2023-01-14 09:31:03,2023-01-14,2023-01-14
111,1130620140-001,-76.5290,3.47147,2023-02-01 17:30:29,NaT,2023-02-01,2023-02-01
112,1130620140-002,-76.5325,3.462316,2023-02-02 14:58:25,NaT,2023-02-02,2023-02-02
113,1002800026-001,-74.1077,4.612405,2023-02-08 14:29:46,2023-02-08 14:29:55,2023-02-08,2023-02-08


In [8]:
df_visitas.shape

(11516, 7)

In [9]:
# base_clientes_cruze  = base_clientes_rutero.copy()
# base_clientes_cruze  = base_clientes_cruze.iloc[:,[3,15,16]]

base_clientes_cruze  = df_clientes.copy()
# base_clientes_cruze  = base_clientes_cruze.iloc[:,[0,1,2,4,5,6,7,8,9,10,11,3,15,16]]
# base_clientes_cruze.drop_duplicates(inplace=True)
# base_clientes_cruze.rename(columns={'LONGITUD': 'LONGITUD_CLI','LATITUD':'LATITUD_CLI'}, inplace=True)
base_clientes_cruze.head()

,COD_CIA,ROWID_TERCERO,NIT_TERCERO,ID_TERCERO,RAZON_SOCIAL,COD_SUCURSAL_CLIENTE,DES_SUCURSAL_CLIENTE,IND_ESTADO_ACTIVO_CLIENTE,COD_DIA_VISITA,DES_DIA_VISITA,COD_CO,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,COD_VENDEDOR,DES_VENDEDOR,COD_CANAL,LONGITUD,LATITUD
93,2,42780.0,15017227,15017227-001,SAKER PADILLA BORIS,001,MINIMERCADO RESTAURANTE MI VAQUITA,1,002,Lunes,005,001,Semanal,0525,VEND MARTINEZ GARCIA EYLEEN,OFF,-75.4696,10.384610
97,2,42793.0,15425449,15425449-001,GARCIA OSPINA HERNAN DE JESUS,001,RESTAURANTE MANANTIAL,1,003,Martes,001,002,Quincena 1 y 3,0136,VEND PABON URIBE LUZ ESTELLA,ON,-75.3739,6.154224
108,2,42868.0,17634873,17634873-004,VARGAS RINCON JOSE,004,AUTOSERVICIO COLSITIENDAS 2,1,004,Miercoles,003,003,Quincena 2 y 4,0325,VEND ACOSTA BENITEZ JAIME ALEXANDER,OFF,-74.0983,4.707879
109,2,42873.0,18501933,18501933-001,OSORIO RAMIREZ REINEL,001,LA ESTANCIA RESTAURANTE BAR,1,005,Jueves,006,002,Quincena 1 y 3,0606,VEND ESCOBAR TAPASCO JOSE OVIDIO,ON,-75.6755,4.814644
143,2,43024.0,22976988,22976988-001,SALCEDO DE CASTRO DELCIRA DE JESUS,001,RANCHO Y LICORES EL TURISTA,1,002,Lunes,005,001,Semanal,0524,VEND MEZA RODRIGUEZ KELLY JOHANA,OFF,-75.5477,10.419420


In [10]:
# df_clientes.info()

In [11]:
# base_clientes_rutero.info()

In [12]:
df_visitas.shape

(11516, 7)

In [13]:
merge_cli_cruze = pd.merge(df_visitas, base_clientes_cruze, how="left")
# merge.to_csv('cruce2.csv')
merge_cli_cruze.head()

,ID_TERCERO,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,FECHAS_VISITA,FECHAS_VISITA_V,COD_CIA,ROWID_TERCERO,NIT_TERCERO,...,COD_DIA_VISITA,DES_DIA_VISITA,COD_CO,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,COD_VENDEDOR,DES_VENDEDOR,COD_CANAL,LONGITUD,LATITUD
0,1070597717-001,-74.1781,4.764811,2023-01-04 08:33:43,NaT,2023-01-04,2023-01-04,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1004120709-001,-74.8264,11.03748,2023-01-14 09:29:36,2023-01-14 09:31:03,2023-01-14,2023-01-14,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1130620140-001,-76.5290,3.47147,2023-02-01 17:30:29,NaT,2023-02-01,2023-02-01,2.0,134301.0,1130620140,...,006,Viernes,009,001,Semanal,0920,VEND AGUDELO MONTENEGRO JHON ALEXANDER,OFF,-76.5367,3.444252
3,1130620140-002,-76.5325,3.462316,2023-02-02 14:58:25,NaT,2023-02-02,2023-02-02,2.0,134301.0,1130620140,...,002,Lunes,009,001,Semanal,0920,VEND AGUDELO MONTENEGRO JHON ALEXANDER,OFF,-76.5373,3.438069
4,1002800026-001,-74.1077,4.612405,2023-02-08 14:29:46,2023-02-08 14:29:55,2023-02-08,2023-02-08,2.0,593653.0,1002800026,...,000,No determinado,007,000,No determinado,0611,VEND RIVERA BERMUDEZ LUIS ALEJANDRO,ON,-75.5677,4.640413


In [14]:
merge_cli_cruze.shape

(11516, 24)

In [15]:
# Convert the datetime columns to datetime objects
merge_cli_cruze['StartDatetime'] = pd.to_datetime(merge_cli_cruze['Inicio_Visita'], format='%m/%d/%Y %H:%M')
merge_cli_cruze['EndDatetime']   = pd.to_datetime(merge_cli_cruze['Finalizacion_Visita'], format='%m/%d/%Y %H:%M')
# df_result['Inicio_Visita_F'] = df_result['Inicio_Visita'].dt.date
# Calculate the time difference
merge_cli_cruze['TimeDifference'] = merge_cli_cruze['EndDatetime'] - merge_cli_cruze['StartDatetime']
# Extract hours and minutes and format as a single column
merge_cli_cruze['TimeDifferenceD'] = merge_cli_cruze['TimeDifference'].apply(lambda x: f"{x.seconds//3600}h:{x.seconds%3600//60} min")
merge_cli_cruze['Time'] = merge_cli_cruze['TimeDifference'].apply(lambda x: f"{x.seconds//3600}{x.seconds%3600//60}")
merge_cli_cruze.drop(['StartDatetime','EndDatetime'],axis=1, inplace=True)
merge_cli_cruze['TimeDifference'] = ['' if x == 'nanh:nan min' else x for x in merge_cli_cruze['TimeDifference']]
merge_cli_cruze['Time']   = pd.to_numeric(merge_cli_cruze['Time'], errors='coerce')

In [16]:
merge_cli_cruze               = merge_cli_cruze.sort_values(by=['ID_TERCERO','FECHAS_VISITA'], ascending=[True, False])
merge_cli_cruze['Logintud']   = pd.to_numeric(merge_cli_cruze['Logintud'], errors='coerce')
merge_cli_cruze['Latitud']    = pd.to_numeric(merge_cli_cruze['Latitud'], errors='coerce')

In [17]:
# Fórmula Haversine
def haversine(lon1, lat1, lon2, lat2):
 # Radio de la Tierra en kilómetros
  R = 6371.0
   # Convierte las coordenadas de grados a radianes
  lat1 = math.radians(lat1)
  lon1 = math.radians(lon1)
  lat2 = math.radians(lat2)
  lon2 = math.radians(lon2)
  # Diferencias de latitud y longitud
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  #Se usa la ecuación "a = [sin ² (dlat / 2) + cos (lat1)] x cos (lat2) x sin ² (dlong / 2)"
  a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  #Distancia en metros
  distance = R * c *1000
  return distance

In [18]:
#Arreglo para almacenar la distancia entre los puntos
DistanciasHaversine =[]
# Recorre las filas del DataFrame
for indice, fila in merge_cli_cruze.iterrows():
  lon1 = fila['Logintud']
  lat1 = fila['Latitud']
  lon2 = fila['LONGITUD']
  lat2 = fila['LATITUD']
  #print(fila)
  if(lon1 != 0 and lon2 != 0 and lat1 != 0 and lat2 != 0):
    total_H = haversine(lon1, lat1, lon2, lat2)
    DistanciasHaversine.append(total_H)
  else:
    DistanciasHaversine.append(0)

merge_cli_cruze["Distancia_Haversine"] = DistanciasHaversine


In [19]:
merge_cli_cruze['Rango_Visita'] = ['Dentro del Rango' if score <= 200 else ('Fuera del Rango' if score >= 200 else '') for score in merge_cli_cruze['Distancia_Haversine']]

In [20]:
merge_cli_cruze.to_csv('cruce2.csv')

In [21]:
merge_cli_cruze.head(3)

,ID_TERCERO,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,FECHAS_VISITA,FECHAS_VISITA_V,COD_CIA,ROWID_TERCERO,NIT_TERCERO,...,COD_VENDEDOR,DES_VENDEDOR,COD_CANAL,LONGITUD,LATITUD,TimeDifference,TimeDifferenceD,Time,Distancia_Haversine,Rango_Visita
11498,1000007753-001,-74.2329,11.116940,2023-09-30 10:09:34,2023-09-30 10:10:06,2023-09-30,2023-09-30,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,0 days 00:00:32,0h:0 min,0.0,711555.991156,Fuera del Rango
10644,1000007753-001,-74.2120,4.717820,2023-09-18 09:50:53,2023-09-18 10:02:28,2023-09-18,2023-09-18,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,0 days 00:11:35,0h:11 min,11.0,11.373101,Dentro del Rango
8224,1000007753-001,-74.2275,4.714956,2023-08-10 12:22:22,NaT,2023-08-10,2023-08-10,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,NaT,nanh:nan min,NaN,1757.393519,Fuera del Rango


In [22]:
# merge_cli_cruze.info()

In [23]:
# base_clientes_rutero.info()

In [24]:
merge_cli_cruze.shape

(11516, 29)

In [25]:
# merge_cli_cruze = merge_cli_cruze.iloc[:,[0,1,2,3,4,5,8,9,10]]
# merge_cli_cruze.drop(['LONGITUD','LATITUD'], axis=1, inplace=True)

In [26]:
merge_cli_cruze.head()

,ID_TERCERO,Logintud,Latitud,Inicio_Visita,Finalizacion_Visita,FECHAS_VISITA,FECHAS_VISITA_V,COD_CIA,ROWID_TERCERO,NIT_TERCERO,...,COD_VENDEDOR,DES_VENDEDOR,COD_CANAL,LONGITUD,LATITUD,TimeDifference,TimeDifferenceD,Time,Distancia_Haversine,Rango_Visita
11498,1000007753-001,-74.2329,11.116940,2023-09-30 10:09:34,2023-09-30 10:10:06,2023-09-30,2023-09-30,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,0 days 00:00:32,0h:0 min,0.0,711555.991156,Fuera del Rango
10644,1000007753-001,-74.2120,4.717820,2023-09-18 09:50:53,2023-09-18 10:02:28,2023-09-18,2023-09-18,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,0 days 00:11:35,0h:11 min,11.0,11.373101,Dentro del Rango
8224,1000007753-001,-74.2275,4.714956,2023-08-10 12:22:22,NaT,2023-08-10,2023-08-10,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,NaT,nanh:nan min,NaN,1757.393519,Fuera del Rango
8036,1000007753-001,-74.2300,4.706296,2023-08-06 00:10:55,NaT,2023-08-06,2023-08-06,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,NaT,nanh:nan min,NaN,2378.823575,Fuera del Rango
8031,1000007753-001,-74.2303,4.706327,2023-08-04 21:55:34,NaT,2023-08-04,2023-08-04,2.0,580135.0,1000007753,...,0304,VEND MARTINEZ BARRETO CARLOS ANDRES,OFF,-74.2119,4.717797,NaT,nanh:nan min,NaN,2405.093001,Fuera del Rango


In [27]:
result = pd.merge(base_clientes_rutero, merge_cli_cruze, how="outer")

In [28]:
result.to_csv('result_1.csv')

In [29]:
# result_example = result[result["ID_TERCERO"]=="51989441-001"] 
#Caso # 1
# 901116026-001
#Caso # 2
# 1000007753-001

In [30]:
result_example  = result.sort_values(by=['ID_TERCERO', 'FECHAS_VISITA'], ascending=[True, True])

In [31]:
# result_example.info()


In [32]:
result_example['Flag_Time'] = result_example.groupby(['ID_TERCERO', 'FECHAS_VISITA'])['Time'].rank(method='dense', ascending=False) # Ok
result_example = result_example.sort_values(by=['ID_TERCERO', 'FECHAS_VISITA','Distancia_Haversine', 'Flag_Time'], ascending=[True, True, True, True])
result_example['Flag_Distance'] = result_example.groupby(['ID_TERCERO', 'FECHAS_VISITA'])['Distancia_Haversine'].rank(method='min', ascending=False) # Ok
# result_example['Flag_Distance'] = result_example.groupby(['ID_TERCERO', 'FECHAS_VISITA'])['Distancia_Haversine','Flag_Time'].rank(method='dense', ascending=True)

# result_example['Flag_DAN'] = [0 if visita is None else 1 for visita in result_example['Inicio_Visita']]
# result_example['Flag_DAN'] = result_example["Inicio_Visita"].isna().astype(int)


In [33]:
def mark_based_on_inicio_visita(row):
    return 0 if pd.isnull(row['Inicio_Visita']) else 1

In [34]:
result_example['Flag_DAN'] = result_example.apply(mark_based_on_inicio_visita, axis=1)

In [35]:
result_example['Flag_Visita_Rutero'] = (result_example['FECHAS_VISITA'] == result_example['FECHAS_VISITA_V']).astype(int)


In [36]:
# result_example.info()

In [37]:
# result_example.head(30)

In [38]:
result_example.shape

(569125, 33)

In [39]:
# Filter rows where Rank equals 1 (top-ranked rows)
# filtered_df = result_example[result_example['Rank'] == 1]

In [40]:
# result_example = result_example[result_example["ID_TERCERO"]=="1006876282-001"] 

In [41]:
result_example = result_example[result_example['ROWID_TERCERO'].notnull()]

In [42]:
# Define a function to remove diacritical marks and special characters
def remove_diacritical_marks(text):
    # Normalize the text to decompose diacritical marks
    normalized_text = unicodedata.normalize('NFKD', text)
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', normalized_text)
    return cleaned_text
    # Use unicodedata to remove diacritical marks from characters
    # text_without_marks = ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])
    # Remove commas from the resulting text
    # return text_without_marks.replace(',', '')


In [43]:
result_example['RAZON_SOCIAL'] = result_example['RAZON_SOCIAL'].apply(remove_diacritical_marks)
result_example['DES_SUCURSAL_CLIENTE'] = result_example['DES_SUCURSAL_CLIENTE'].apply(remove_diacritical_marks)
result_example['DES_DIA_VISITA'] = result_example['DES_DIA_VISITA'].apply(remove_diacritical_marks)
result_example['DES_VENDEDOR'] = result_example['DES_VENDEDOR'].apply(remove_diacritical_marks)

In [44]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# df_data = result_example[result_example['DES_DIA_VISITA']=='Sábado']
# df_data

In [45]:
# Calculate the number of rows in each part
num_rows = len(result_example)
num_parts = 3
rows_per_part = num_rows // num_parts
remainder = num_rows % num_parts  # Handles the remainder of rows

# Initialize a list to store the parts
parts = []

# Split the DataFrame into 5 parts
start_idx = 0
for i in range(num_parts):
    if i < remainder:
        end_idx = start_idx + rows_per_part + 1
    else:
        end_idx = start_idx + rows_per_part
    
    part_df = result_example.iloc[start_idx:end_idx]
    parts.append(part_df)
    
    start_idx = end_idx

# Save each part as a separate CSV file
for i, part_df in enumerate(parts):
    part_df.to_csv(f'result_example_client_{i+1}.csv', sep=';', encoding='latin1', errors='ignore', index=False)

In [46]:
# num_rows = len(result_example)
# rows_per_part = num_rows // 10

# # Split the DataFrame into 5 parts
# parts = []
# for i in range(10):
#     start_idx = i * rows_per_part
#     end_idx = (i + 1) * rows_per_part if i < 9 else num_rows
#     part_df = result_example.iloc[start_idx:end_idx]
#     parts.append(part_df)

# # Save each part as a separate CSV file
# for i, part_df in enumerate(parts):
#     part_df.to_csv(f'result_example_client_{i+1}.csv', index=False)

In [47]:
with open('/home/pydev/workflow/dt_unifed_statistics/src/datos_con_caracteres_especiales.csv', 'w', encoding='latin1') as f:     
    result_example.to_csv(f, index=False)

In [48]:
# result_example.to_csv('result_example_client.csv', sep=',', encoding="utf-8", index=False)

In [49]:
# result_example.to_xml('data.xml')

In [50]:
# result_example.to_parquet('result_example_client.parquet')

In [51]:
# base_clientes_rutero_2 = base_clientes_rutero.copy()
# base_clientes_rutero_2 = base_clientes_rutero_2.iloc[:, [3, 18]]
# base_clientes_rutero_2.rename(columns={'FECHAS_VISITA': 'FECHA_VISTA_CRUCE'}, inplace=True)


In [52]:
result_example.head()

,COD_CIA,ROWID_TERCERO,NIT_TERCERO,ID_TERCERO,RAZON_SOCIAL,COD_SUCURSAL_CLIENTE,DES_SUCURSAL_CLIENTE,IND_ESTADO_ACTIVO_CLIENTE,COD_DIA_VISITA,DES_DIA_VISITA,...,FECHAS_VISITA_V,TimeDifference,TimeDifferenceD,Time,Distancia_Haversine,Rango_Visita,Flag_Time,Flag_Distance,Flag_DAN,Flag_Visita_Rutero
16296,2.0,309895.0,0121,0121-001,VEND AUTOV ZONA VACANTE 0121,001,VEND AUTOV COSSIO ECHAVARRIA LUIS JAVIER,1.0,002,Lunes,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0,0
16297,2.0,309895.0,0121,0121-001,VEND AUTOV ZONA VACANTE 0121,001,VEND AUTOV COSSIO ECHAVARRIA LUIS JAVIER,1.0,002,Lunes,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0,0
16298,2.0,309895.0,0121,0121-001,VEND AUTOV ZONA VACANTE 0121,001,VEND AUTOV COSSIO ECHAVARRIA LUIS JAVIER,1.0,002,Lunes,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0,0
16299,2.0,309895.0,0121,0121-001,VEND AUTOV ZONA VACANTE 0121,001,VEND AUTOV COSSIO ECHAVARRIA LUIS JAVIER,1.0,002,Lunes,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0,0
16300,2.0,309895.0,0121,0121-001,VEND AUTOV ZONA VACANTE 0121,001,VEND AUTOV COSSIO ECHAVARRIA LUIS JAVIER,1.0,002,Lunes,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [53]:
result.shape

(569125, 29)

In [54]:
base_clientes_rutero.shape

(561325, 19)

In [55]:
base_clientes_rutero.head()

,COD_CIA,ROWID_TERCERO,NIT_TERCERO,ID_TERCERO,RAZON_SOCIAL,COD_SUCURSAL_CLIENTE,DES_SUCURSAL_CLIENTE,IND_ESTADO_ACTIVO_CLIENTE,COD_DIA_VISITA,DES_DIA_VISITA,COD_CO,COD_FRECUENCIA_VISITA,DES_FRECUENCIA_VISITA,COD_VENDEDOR,DES_VENDEDOR,COD_CANAL,LONGITUD,LATITUD,FECHAS_VISITA
0,2,42780.0,15017227,15017227-001,SAKER PADILLA BORIS,001,MINIMERCADO RESTAURANTE MI VAQUITA,1,002,Lunes,005,001,Semanal,0525,VEND MARTINEZ GARCIA EYLEEN,OFF,-75.4696,10.38461,2023-01-02
1,2,42780.0,15017227,15017227-001,SAKER PADILLA BORIS,001,MINIMERCADO RESTAURANTE MI VAQUITA,1,002,Lunes,005,001,Semanal,0525,VEND MARTINEZ GARCIA EYLEEN,OFF,-75.4696,10.38461,2023-01-09
2,2,42780.0,15017227,15017227-001,SAKER PADILLA BORIS,001,MINIMERCADO RESTAURANTE MI VAQUITA,1,002,Lunes,005,001,Semanal,0525,VEND MARTINEZ GARCIA EYLEEN,OFF,-75.4696,10.38461,2023-01-16
3,2,42780.0,15017227,15017227-001,SAKER PADILLA BORIS,001,MINIMERCADO RESTAURANTE MI VAQUITA,1,002,Lunes,005,001,Semanal,0525,VEND MARTINEZ GARCIA EYLEEN,OFF,-75.4696,10.38461,2023-01-23
4,2,42780.0,15017227,15017227-001,SAKER PADILLA BORIS,001,MINIMERCADO RESTAURANTE MI VAQUITA,1,002,Lunes,005,001,Semanal,0525,VEND MARTINEZ GARCIA EYLEEN,OFF,-75.4696,10.38461,2023-01-30


In [56]:
import pandas as pd
import unicodedata
import re

# Create a sample DataFrame
data = {'text_column': ['Ñató, Sábado, Nuñez.', '@LICORES.COM']}
df = pd.DataFrame(data)

# Define a function to remove diacritical marks and special characters
def remove_diacritical_marks(text):
    # Normalize the text to decompose diacritical marks
    normalized_text = unicodedata.normalize('NFKD', text)
    # Use a regular expression to remove non-alphanumeric characters
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', normalized_text)
    return cleaned_text

# Apply the function to the DataFrame column
df['text_column'] = df['text_column'].apply(remove_diacritical_marks)

# Display the updated DataFrame
print(df)


         text_column
0  Nato Sabado Nunez
1         LICORESCOM


In [57]:
import pandas as pd
import unicodedata

# Create a sample DataFrame
data = {'text_column': ['Ñató, Sábado, Nuñez.','@LICORES.COM']}
df = pd.DataFrame(data)

# Define a function to remove diacritical marks from characters
def remove_diacritical_marks(text):
    # Use unicodedata to remove diacritical marks from characters
    return ''.join([c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)])

# Apply the function to the DataFrame column
df['text_column'] = df['text_column'].apply(remove_diacritical_marks)

# Display the updated DataFrame
print(df)


            text_column
0  Nato, Sabado, Nunez.
1          @LICORES.COM
